In [ ]:
!pip install --quiet pytorch-lightning

Exception ignored in: <function _get_module_lock.<locals>.cb at 0x7fc156da71f0>
Traceback (most recent call last):
  File "<frozen importlib._bootstrap>", line 182, in cb
KeyboardInterrupt: 
Traceback (most recent call last):
  File "/usr/local/bin/pip3", line 5, in <module>
    from pip._internal.cli.main import main
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/cli/main.py", line 9, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/cli/autocompletion.py", line 10, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/cli/main_parser.py", line 8, in <module>
    from pip._internal.cli import cmdoptions
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/cli/cmdoptions.py", line 23, in <module>
    from pip._internal.cli.parser import ConfigOptionParser
  File "/usr/local/lib/python3.8/dist-packa

In [ ]:
import tensorflow as tf
import json
import os
import cv2
from torch.nn import functional as F
from pathlib import Path
import torchvision
from google.colab.patches import cv2_imshow
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from pathlib import Path
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
import timm
from torch import nn
import PIL
import torchvision.models as models

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from transformers import (AdamW,T5ForConditionalGeneration, AutoTokenizer as Tokenizer)

In [ ]:
pl.seed_everything(42)

INFO:lightning_fabric.utilities.seed:Global seed set to 42


42

In [ ]:
_URL = 'http://images.cocodataset.org/zips/val2014.zip'
zip_dir = tf.keras.utils.get_file('/content/MSCOCOVAL2014.zip', origin=_URL, extract=False,archive_format='auto')
fname = '/content/MSCOCOVAL2014.zip'
!unzip -q $fname -d /content/

replace /content/val2014/COCO_val2014_000000324670.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
_URL = 'https://s3.amazonaws.com/cvmlp/vqa/mscoco/vqa/v2_Questions_Val_mscoco.zip'
zip_dir = tf.keras.utils.get_file('/content/QUESVAL2014.zip', origin=_URL, extract=False,archive_format='auto')
fname = '/content/QUESVAL2014.zip'
!unzip -q $fname -d /content/

replace /content/v2_OpenEnded_mscoco_val2014_questions.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
_URL = 'https://s3.amazonaws.com/cvmlp/vqa/mscoco/vqa/v2_Annotations_Val_mscoco.zip'
zip_dir = tf.keras.utils.get_file('/content/ANNOTVAL2014.zip', origin=_URL, extract=False,archive_format='auto')
fname = '/content/ANNOTVAL2014.zip'
!unzip -q $fname -d /content/

replace /content/v2_mscoco_val2014_annotations.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
with open(os.path.join('/content/', 'v2_OpenEnded_mscoco_val2014_questions.json'), 'r') as f:
    val_questions = json.load(f)['questions']
with open(os.path.join('/content/', 'v2_mscoco_val2014_annotations.json'), 'r') as f:
    val_answers = json.load(f)['annotations']

In [ ]:
val_data = []
for question, annotation in zip(val_questions, val_answers):
    question_text = question['question']
    image_id = annotation['image_id']
    answer = annotation['answers'][0]['answer']
    image_filename = 'COCO_val2014_{:012d}.jpg'.format(image_id)
    image_path = os.path.join('/content/', 'val2014', image_filename)
    val_data.append({'question': question_text, 'image_path': image_path, 'answer': answer})

In [ ]:
# Convert the array of dictionaries to a DataFrame
df = pd.DataFrame(val_data)

In [ ]:
def has_three_channels(image_path):
    with PIL.Image.open(image_path) as img:
        return img.mode == 'RGB'

# Filter the DataFrame to keep only the images with 3 channels
df = df[df['image_path'].apply(has_three_channels)]

In [ ]:
def show_sample(idx=0):
  print("Q : ",df.iloc[idx]['question'])
  image = cv2.imread(df.iloc[idx]['image_path'])
  image = cv2.resize(image, (224, 224))  
  cv2_imshow(image)
  print("A : ",df.iloc[idx]['answer'])

In [ ]:
show_sample(100)

# **Model**

In [ ]:
df = df[:5000]

In [ ]:
train_df, val_df = train_test_split(df,test_size=0.1)

In [ ]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

class ImageDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx]['image_path']
        image = Image.open(img_path)
        if self.transform:
            image = self.transform(image)
        
        return dict(img = image)

class ImageDataModule(pl.LightningDataModule):
    def __init__(self, train_df, val_df, batch_size=8, num_workers=4, transform=None):
        super().__init__()
        self.train_df = train_df
        self.val_df = val_df
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.transform = transform

    def setup(self, stage=None):
        self.train_dataset = ImageDataset(self.train_df, transform=self.transform)
        self.val_dataset = ImageDataset(self.val_df, transform=self.transform)

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, num_workers=self.num_workers, shuffle=True, pin_memory=True)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, num_workers=self.num_workers, pin_memory=True)

    def test_dataloader(self):
        return None # No test dataset


In [ ]:
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [ ]:
class Generator(nn.Module):
    def __init__(self, latent_dim: int):
        super().__init__()
        self.latent_dim = latent_dim
        self.fc = nn.Linear(latent_dim, 256 * 8 * 8)
        self.conv1 = nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1)
        self.conv2 = nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1)
        self.conv3 = nn.ConvTranspose2d(64, 3, kernel_size=4, stride=2, padding=1)

    def forward(self, z):
        x = self.fc(z)
        x = x.view(-1, 256, 8, 8)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = torch.tanh(self.conv3(x))
        return x

class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1)
        self.fc = nn.Linear(256 * 8 * 8, 1)

    def forward(self, x):
        x = F.leaky_relu(self.conv1(x), 0.2)
        x = F.leaky_relu(self.conv2(x), 0.2)
        x = F.leaky_relu(self.conv3(x), 0.2)
        x = x.view(-1, 256 * 8 * 8)
        x = torch.sigmoid(self.fc(x))
        return x

In [ ]:
class GAN(pl.LightningModule):
    def __init__(self, latent_dim: int = 64, lr: float = 0.0002, b1: float = 0.5, b2: float = 0.999):
        super().__init__()
        self.save_hyperparameters()
        self.generator = Generator(latent_dim)
        self.discriminator = Discriminator()

    def forward(self, z):
        return self.generator(z)

    def training_step(self, batch, batch_idx, optimizer_idx):
        real_images = batch['img']
        batch_size = real_images.size(0)
        device = real_images.device
        valid = torch.ones(batch_size, 1, device=device)
        fake = torch.zeros(batch_size, 1, device=device)

        if optimizer_idx == 0: # Train generator
            z = torch.randn(batch_size, self.hparams.latent_dim, device=device)
            gen_images = self.generator(z)
            g_loss = F.binary_cross_entropy(self.discriminator(gen_images), valid)
            self.log('g_loss', g_loss)
            return g_loss
        else: # Train discriminator
            z = torch.randn(batch_size, self.hparams.latent_dim, device=device)
            gen_images = self.generator(z).detach()

            real_loss = F.binary_cross_entropy(self.discriminator(real_images), valid)
            fake_loss = F.binary_cross_entropy(self.discriminator(gen_images), fake)
            d_loss = (real_loss + fake_loss) / 2
            self.log('d_loss', d_loss)
            return d_loss

    def validation_step(self, batch, batch_idx):
        real_images = batch['img']
        batch_size = real_images.size(0)
        device = real_images.device
        valid = torch.ones(batch_size, 1, device=device)
        fake = torch.zeros(batch_size, 1, device=device)

        z = torch.randn(batch_size, self.hparams.latent_dim, device=device)
        gen_images = self.generator(z).detach()

        real_loss = F.binary_cross_entropy(self.discriminator(real_images), valid)
        fake_loss = F.binary_cross_entropy(self.discriminator(gen_images), fake)
        d_loss = (real_loss + fake_loss) / 2
        self.log('val_loss', d_loss, prog_bar=True)

    def configure_optimizers(self):
        lr = self.hparams.lr
        betas = (self.hparams.b1, self.hparams.b2)
        opt_g = torch.optim.Adam(self.generator.parameters(), lr=lr, betas=betas)
        opt_d = torch.optim.Adam(self.discriminator.parameters(), lr=lr, betas=betas)
        return [opt_g, opt_d], []

    def train_epoch_end(self):
        z = torch.randn(8, self.hparams.latent_dim, device=self.device)
        sample_imgs = self.generator(z)
        grid = torchvision.utils.make_grid(sample_imgs, nrow=4, normalize=True)
        self.logger.experiment.add_image('generated_images', grid, global_step=self.current_epoch)

    def validation_epoch_end(self, outputs):
        if len(outputs) > 0:
          avg_val_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
          self.log('val_loss', avg_val_loss, prog_bar=True)


In [ ]:
BATCH_SIZE = 8
N_EPOCHS = 10

In [ ]:
data_module = ImageDataModule(train_df, val_df, batch_size=BATCH_SIZE, 
                              num_workers=4, transform=transform)
data_module.setup()

In [ ]:
model = GAN()

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath = 'checkpoints',
    filename = 'best_cp',
    save_top_k = 1,
    verbose = True,
    monitor = 'val_loss',
    mode = 'min')

In [ ]:
trainer = pl.Trainer(
    callbacks=[checkpoint_callback],
    max_epochs = N_EPOCHS)

In [ ]:
trainer.fit(model,data_module)

In [ ]:
from PIL import Image

# Define the transformation to apply on the input image
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

# Load the image from file
image = Image.open("/content/val2014/COCO_val2014_000000000042.jpg")
cv2_imshow(image)

# Apply the transformation
image = transform(image)

# Load the saved checkpoint
checkpoint_path = "/content/checkpoints/best_cp.ckpt"
model = GAN.load_from_checkpoint(checkpoint_path)

# Put the model in evaluation mode
model.eval()

# Generate the image
with torch.no_grad():
    z = torch.randn(1, model.hparams.latent_dim, device=model.device)
    gen_image = model.generator(z)
    gen_image = (gen_image + 1) / 2  # Convert the output to a valid image range (0, 1)

# Display the generated image
import matplotlib.pyplot as plt
plt.imshow(gen_image[0].permute(1, 2, 0).cpu())
plt.show()
